In [1]:
include("../LiPoSID.jl")
include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials

using Dates
using Statistics

Consider Kossakowski constraints [1] for two-level system (Theorem 3.1 on page 824).

### References

1. **Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan (1976)** "Completely positive dynamical semigroups of Nlevel systems" *J. Math. Phys.* 17, 821 (1976) DOI: [10.1063/1.522979](https://doi.org/10.1063/1.522979)

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2; fᴷ₂ = σʸ/2; fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]+0.0
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 

kossak_constrs = [κ₁, κ₂, κ₃, γ[1], γ[2], γ[3], constr1, constr2, constr3]

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

Hˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

Hᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(Hᴷˢʸᵐᵇ) == 0
@assert Hᴷˢʸᵐᵇ == Hˢʸᵐᵇ

### Auxilary functions

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

If we need fidelity more then certain threshold $F$ we have constraint:

$4\det(\rho) \det(\sigma) - F^2 + 2 F \operatorname{tr}(\rho \sigma) - \operatorname{tr}(\rho \sigma)^2 > 0$

It is more convenient to use infidelity therhold $\Delta = 1-F \ll 1$

In [3]:
function infidelity_constr(ρ, σ, Δ) #for two-level system
    F = 1-Δ
    return 4*(det(ρ)*det(σ)) - F^2 + 2*F*tr(ρ*σ) - tr(ρ*σ)^2
end

function infidelity_constr(ρ, σ) #for two-level system
    return 4*(det(ρ)*det(σ)) - 1 + 2*tr(ρ*σ) - tr(ρ*σ)^2
end

infidelity_constr (generic function with 2 methods)

For fidelity $F=1$ we can write equality constraint:

$1 = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

$(1 - \operatorname{tr}(\rho \sigma))^2 = 4\det(\rho) \det(\sigma)$

$(1 - \operatorname{tr}(\rho \sigma))^2 - 4\det(\rho) \det(\sigma) = 0$

In [4]:
function infidelity_equality_constr(ρ, σ) #for two-level system
    return 1 - 2*tr(ρ*σ) + tr(ρ*σ)^2 - 4*(det(ρ)*det(σ)) 
end

infidelity_equality_constr (generic function with 1 method)

Right hand side of the GKSL equation in GKS form:

$\frac{d\rho}{dt} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where $C \succeq 0$ - semidefinite matrix

In [5]:
function Dc(ρ, t)
    U = (Hˢʸᵐᵇ*ρ - ρ*Hˢʸᵐᵇ)/im 
    D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ])/2
    return U + D
end 

Dc (generic function with 1 method)

In [6]:
function reassemble_poly(p)
    if isempty(monomials(p))
        p = 0. 
    else
        poly = sum(real(p) * mon for (coef, mon) in zip(coefficients(p), monomials(p)))
    end
    return poly
end

reassemble_poly (generic function with 1 method)

In [11]:
function kossak_obj_infid_constrs(ρ, t)

    constrs = []
    obj = 0
    δ = 1e-3 # infidelity threshold
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i],t[i]) - 4*Dc(ρ[i-1],t[i-1]) - Dc(ρ[i-2],t[i-2])) / 3
        obj += LiPoSID.frobenius_norm2(ρ1-ρ2)
        #push!(constrs, reassemble_poly(infidelity_constr(ρ1, ρ2, δ)))
        push!(constrs, reassemble_poly(infidelity_constr(ρ1, ρ2)))
    end

    obj = reassemble_poly(obj)

    return obj, constrs

end

kossak_obj_infid_constrs (generic function with 1 method)

Read Kurt's spin-boson system data 

In [12]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

Assemble objective and constraints using Kossakowski and fidelity constraints

In [13]:
function kossak_GEXY_obj_infid_constrs(ρᵍᵉˣʸ, tᵍᵉˣʸ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG, constrsG = kossak_obj_infid_constrs(ρᵍ, tᵍ)
    polyE, constrsE  = kossak_obj_infid_constrs(ρᵉ, tᵉ)
    polyX, constrsX  = kossak_obj_infid_constrs(ρˣ, tˣ)
    polyY, constrsY  = kossak_obj_infid_constrs(ρʸ, tʸ)

    polyGEXY = polyG + polyE + polyX + polyY
    constrsGEXY = [constrsG, constrsE, constrsX, constrsY]

    return polyGEXY, constrsGEXY 
end

kossak_GEXY_obj_infid_constrs (generic function with 1 method)

Load data

In [14]:
file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(file_name, γᵢ);

Assemble objective and constraints

In [15]:
polyGEXY, constrsGEXY = kossak_GEXY_obj_infid_constrs(ρᵍᵉˣʸ, tᵍᵉˣʸ)

(0.023368540768171964γ₁⁴ + 0.0230556033255625γ₁³a₁ + 0.02360289689036557γ₁³ϵ + 0.023388792778359008γ₁³h_Im + 0.04675424025623273γ₁²γ₂² + 0.02278145386346557γ₁²γ₂a₂ + 0.023134184645978363γ₁²γ₂ϵ + 0.02338202739044469γ₁²γ₂h_Re + 0.4678554545005836γ₁²γ₃² + 0.9623939233196066γ₁²γ₃a₃ + 0.02335505414589924γ₁²γ₃h_Re + 0.023348288757984927γ₁²γ₃h_Im + 0.5873617761707242γ₁²a₁² + 0.023368540768171964γ₁²a₁a₂ + 0.023368540768171964γ₁²a₁a₃ + 0.023877046352462496γ₁²a₁ϵ + 0.9621012378871843γ₁²a₁h_Im + 0.5876747136133336γ₁²a₂² + 0.023368540768171964γ₁²a₂a₃ + 0.0230556033255625γ₁²a₂ϵ - 0.9156568417832628γ₁²a₂h_Re + 0.5876747136133336γ₁²a₃² + 0.02278145386346557γ₁²a₃h_Re + 0.02368147821078143γ₁²a₃h_Im + 0.07035713714659829γ₁²ϵ² + 0.023348288757984927γ₁²ϵh_Re + 0.023636635522825335γ₁²ϵh_Im + 0.4912411539886441γ₁²h_Re² + 0.02360289689036557γ₁²h_Reh_Im + 0.49124424727894245γ₁²h_Im² + 0.0230727620454513γ₁γ₂²a₁ + 0.02362005561025437γ₁γ₂²ϵ + 0.023405951498247805γ₁γ₂²h_Im - 0.0009000243473158575γ₁γ₂a₁a₂ - 0.0005

In [16]:
@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))
maxdegree(polyGEXY)

minimum(abs.(coefficients(polyGEXY))) = 1.0842021724855044e-19
maximum(abs.(coefficients(polyGEXY))) = 69.01175526318613


4

In [17]:
function next_larger_index(arr::Vector{T}, value::T) where T
    n = length(arr)
    if n == 0
        error("Array must not be empty")
    end

    spacing = arr[2] - arr[1]
    approximate_index = ceil(Int, (value - arr[1]) / spacing) + 1

    # Ensure the index is within valid range
    if approximate_index > n
        error("No element in the array is larger than the given value")
    end

    return approximate_index
end




next_larger_index (generic function with 1 method)

In [18]:
constrsGEXYsparced = [constrs[1:1000:end] for constrs in constrsGEXY]

gamma_index = next_larger_index(tᵍᵉˣʸ[1], 1/parse(Float64,γᵢ))
single_constraint_GEXY = [constrs[gamma_index] for constrs in constrsGEXY]

4-element Vector{Polynomial{true, Float64}}:
 -1.7064771154581047e-21γ₁⁴ - 1.7064771154337755e-21γ₁³γ₂ + 9.343152774943757e-20γ₁³γ₃ - 3.263439909909636e-14γ₁³a₁ - 1.7065247709443113e-21γ₁³a₂ + 9.380882971378369e-20γ₁³a₃ - 1.836700598954176e-20γ₁³ϵ - 1.7064784107415023e-21γ₁³h_Re - 3.25054875910954e-14γ₁³h_Im - 4.237113353805364e-20γ₁²γ₂² - 9.850432036965411e-18γ₁²γ₂γ₃ - 3.2634399098863727e-14γ₁²γ₂a₁ + 1.5930665266923435e-13γ₁²γ₂a₂ - 9.889490492157048e-18γ₁²γ₂a₃ - 1.7064771148002138e-21γ₁²γ₂ϵ - 1.5867736726862732e-13γ₁²γ₂h_Re - 3.2505487589819614e-14γ₁²γ₂h_Im - 1.9597338432045977e-5γ₁²γ₃² + 8.770677453009259e-13γ₁²γ₃a₁ + 1.9477892880723815e-11γ₁²γ₃a₂ - 3.935011662335255e-5γ₁²γ₃a₃ + 2.578219788449837e-18γ₁²γ₃ϵ + 6.8808956149583e-13γ₁²γ₃h_Re + 4.988897398188463e-12γ₁²γ₃h_Im - 1.5602379110324511e-7γ₁²a₁² - 3.2634854777119625e-14γ₁²a₁a₂ + 8.806754771063114e-13γ₁²a₁a₃ - 1.919410701304619e-13γ₁²a₁ϵ - 3.263441148449467e-14γ₁²a₁h_Re - 3.1081494047269514e-7γ₁²a₁h_Im - 1.5602375847371542e-7γ₁²a₂²

In [19]:
fidelity_constrs = vcat(single_constraint_GEXY...)
length(fidelity_constrs)

4

In [18]:
fidelity_constrs

4-element Vector{Polynomial{true, Float64}}:
 -1.7064771154581047e-21γ₁⁴ - 1.7064771154337755e-21γ₁³γ₂ + 9.343152774943757e-20γ₁³γ₃ - 3.263439909909636e-14γ₁³a₁ - 1.7065247709443113e-21γ₁³a₂ + 9.380882971378369e-20γ₁³a₃ - 1.836700598954176e-20γ₁³ϵ - 1.7064784107415023e-21γ₁³h_Re - 3.25054875910954e-14γ₁³h_Im - 4.237113353805364e-20γ₁²γ₂² - 9.850432036965411e-18γ₁²γ₂γ₃ - 3.2634399098863727e-14γ₁²γ₂a₁ + 1.5930665266923435e-13γ₁²γ₂a₂ - 9.889490492157048e-18γ₁²γ₂a₃ - 1.7064771148002138e-21γ₁²γ₂ϵ - 1.5867736726862732e-13γ₁²γ₂h_Re - 3.2505487589819614e-14γ₁²γ₂h_Im - 1.9597338432045977e-5γ₁²γ₃² + 8.770677453009259e-13γ₁²γ₃a₁ + 1.9477892880723815e-11γ₁²γ₃a₂ - 3.935011662335255e-5γ₁²γ₃a₃ + 2.578219788449837e-18γ₁²γ₃ϵ + 6.8808956149583e-13γ₁²γ₃h_Re + 4.988897398188463e-12γ₁²γ₃h_Im - 1.5602379110324511e-7γ₁²a₁² - 3.2634854777119625e-14γ₁²a₁a₂ + 8.806754771063114e-13γ₁²a₁a₃ - 1.919410701304619e-13γ₁²a₁ϵ - 3.263441148449467e-14γ₁²a₁h_Re - 3.1081494047269514e-7γ₁²a₁h_Im - 1.5602375847371542e-7γ₁²a₂²

In [20]:
final_constrains = [fidelity_constrs..., kossak_constrs...]
length(final_constrains)

13

Optimization

In [21]:
using TSSOS

In [22]:
vars = variables(polyGEXY)
opt,sol,data = cs_tssos_first([polyGEXY, final_constrains...], vars, 4, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.086169584 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 1.15848303 seconds.
The maximal size of blocks is 715.
Assembling the SDP...
There are 24310 affine constraints.
SDP assembling time: 6.092814223 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 24310           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 36      

(0.005266177294555219, [0.32749727382088056, -0.8847930316421199, -0.11284020479313155, -1.5803089073312848, -0.16315044363589384, -0.5033209481056911, -0.21085594631546078, -0.03961062298151998, -1.488508579954141], TSSOS.mcpop_data(9, 0, 13, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0007], [0x0001, 0x0001, 0x0001, 0x0009], [0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0002, 0x0002]  …  [0x0008, 0x0008, 0x0008, 0x0009], [0x0008, 0x0008, 0x0009], [0x0008, 0x0008, 0x0009, 0x0009], [0x0008, 0x0009], [0x0008, 0x0009, 0x0009], [0x0008, 0x0009, 0x0009, 0x0009], [0x0009], [0x0009, 0x0009], [0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009]], [[0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0001, 0x0003], [0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0005], [0x0001, 0x0001, 0x0001, 0x0006], [0x000

In [23]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-1.32513+0.0im)   (-0.0+0.503321im)
 (-0.0-0.503321im)  (1.09945+0.0im)

In [24]:
Hˢⁱᵈ = convert.(ComplexF64,Hˢⁱᵈ)
Cˢⁱᵈ = convert.(ComplexF64,Cˢⁱᵈ)

3×3 Matrix{ComplexF64}:
 -1.32513+0.0im           0.0+0.503321im        0.0-0.16315im
      0.0-0.503321im  1.09945+0.0im             0.0+1.58031im
      0.0+0.16315im       0.0-1.58031im   -0.444456+0.0im

In [25]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

1-element Vector{Any}:
 ComplexF64[-0.37937464739110094 + 0.0im 0.7086331305393299 + 0.028477307578243655im; -0.5289440619296475 + 0.00992628181094752im 0.37937464739110094 + 0.0im]

In [26]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [27]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.14295106716778255
Median fidelity for 0.25133: 0.6243027781553983
